TU Delft internship of Bordeaux ENSEIRB -MATMECA student Anouchka Desmettre 
Dates: June 1st - August 31st, 2024 

Supervision: D. Lahaye (DIAM - EEMCS Faculty - TU Delft)

# Title: Mathematical modelling and numerical simulation of driving buses and vibrations door   

## 5. Stationary Thin Beam Equation - Biharmonic Equation - Deflection 

The bi-harmonic equation in the context of a stationary thin beam typically describes the deflection of the beam. It can be expressed as:

$$ \nabla^4 \Phi = f \iff \nabla^2(\nabla^2 \Phi) = f $$
Here, $\nabla^4$ represents the biharmonic operator, $\Phi$ represents the displacement of the beam under load. The function $f$ on the right-hand side typically corresponds to external loads or forces applied to the beam. This equation provides a comprehensive model for predicting how beams bend and flex under such forces, allowing engineers to design structures that are both strong and efficient.

### 5.1 Analytical Reference Solution

Source : https://en.wikipedia.org/wiki/Fundamental_solution#Biharmonic_equation 

The biharmonic equation can be written as:
$$
-\Delta^2 \Phi(\mathbf{x}, \mathbf{x}') = \delta(\mathbf{x} - \mathbf{x}')
$$
In this equation:
- $\Delta^2$ is the biharmonic operator, which is the square of the Laplacian operator, $\Delta$.
- $\Phi(\mathbf{x}, \mathbf{x}')$ is the fundamental solution.
- $\delta(\mathbf{x} - \mathbf{x}')$ is the Dirac delta function, representing a point source of influence at the position $\mathbf{x}'$.

The fundamental solutions vary depending on the dimension of the space in which the equation is solved:

1. **In 2 dimensions (2D)**:
   $$
   \Phi_{\text{2D}}(\mathbf{x}, \mathbf{x}') = -\frac{|\mathbf{x} - \mathbf{x}'|^2}{8\pi} \ln |\mathbf{x} - \mathbf{x}'|
   $$
   Here, the solution shows that the influence of the point source decreases logarithmically with distance and is modulated by the square of the distance.

2. **In 3 dimensions (3D)**:
   $$
   \Phi_{\text{3D}}(\mathbf{x}, \mathbf{x}') = \frac{|\mathbf{x} - \mathbf{x}'|}{8\pi}
   $$
   This form indicates a decrease in the influence of the source proportional to the distance, a characteristic typical of interactions in 3 dimensions, like gravitational or electrostatic fields in free space.

### 5.2. Shooting Method Implementation

Using : https://github.com/Youkrid/Youkrid.github.io 

1. **What is `BoundaryValueDiffEq`?**

`BoundaryValueDiffEq` is a Julia package specifically designed to solve boundary value problems (BVPs) for ordinary differential equations (ODEs). BVPs require solutions to differential equations where conditions are specified at different points, typically at the domain's boundaries, unlike initial value problems where conditions are only given at the start point.

2. **Role of the Shooting Method in `BoundaryValueDiffEq`**

The shooting method, implemented in the `BoundaryValueDiffEq` package, is one approach to solve BVPs. Here's how it typically works:
- **Initial Guess**: Start with an initial guess for the unknown initial conditions at one boundary.
- **Solve Forward**: Use an ODE solver to integrate the differential equation from one boundary to the other.
- **Adjust Guess**: Adjust the initial conditions based on the difference between the computed endpoint and the specified boundary condition at the final boundary.
- **Iterate**: Repeat the process, refining the initial conditions until the solution converges to satisfy the boundary conditions at both ends.

The shooting method, while useful for solving boundary value problems numerically, does not directly yield the fundamental solution of the biharmonic equation in 2D, such as $\Phi_{\text{2D}}(\mathbf{x}, \mathbf{x}') = -\frac{|\mathbf{x} - \mathbf{x}'|^2}{8\pi} \ln |\mathbf{x} - \mathbf{x}'|$, which is derived using analytical techniques for a point source in an infinite domain.

3. **Implementing the Shooting Method in Code**


### 5.3 Finite Difference Method on Uniform Spatial Mesh

https://github.com/ziolai/finite_element_electrical_engineering/tree/main  

Implementing the finite difference method on a uniform spatial mesh involves discretizing the bi-harmonic equation using second-order central differences (for accuracy) and solving the resulting system of equations iteratively. This method is straightforward to implement and can handle uniform geometries effectively.

### 5.4 Finite Element Method on Non-uniform Spatial Mesh

The finite element method (FEM) on a non-uniform spatial mesh requires discretizing the domain into smaller, irregular elements. This method is more versatile than finite differences for handling complex geometries and material properties but requires careful implementation of basis functions and assembly of stiffness matrices.